In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Import scikit-learn dataset library
from sklearn import datasets

#Load dataset
iris = datasets.load_iris()

# Creating a DataFrame of given iris dataset.
import pandas as pd
data=pd.DataFrame({
    'sepal length':iris.data[:,0],
    'sepal width':iris.data[:,1],
    'petal length':iris.data[:,2],
    'petal width':iris.data[:,3],
    'species':iris.target
})

In [ ]:
###
### Attempt at ugly MixUp version of the above training
### Currently not working: need to do some one-hot version of the labels etc I guess
###

# Import train_test_split function
from sklearn.model_selection import train_test_split

# Convert dataframe to arrays and perform one-hot encoding

X=data[['sepal length', 'sepal width', 'petal length', 'petal width']]  # Features
y=data['species']  # Labels

X = X.to_numpy()

enc = LabelBinarizer()
enc.fit(y)
y = enc.transform(y)

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70
X_train_base = X_train.copy()
y_train_base = y_train.copy()

###
### Create the MixUp data set
# number of MixUp extensions
number_of_mixups = 9
n = np.shape(X_train_base)[0]
for i in range(number_of_mixups):
  shuffled_indices = np.arange(n).tolist()
  np.random.shuffle(shuffled_indices)
  X_s = X_train_base[shuffled_indices]
  y_s = y_train_base[shuffled_indices]
  mixup_l = np.random.beta(0.2,0.2)
  X_mixed = X_train_base*(1-mixup_l) + mixup_l*X_s
  y_mixed = y_train_base*(1-mixup_l) + (mixup_l)*(y_s)
  X_train = np.concatenate([X_train, X_mixed])
  y_train = np.concatenate([y_train, y_mixed])



#Import Random Forest Model
from sklearn.ensemble import RandomForestRegressor
clf=RandomForestRegressor(n_estimators=10)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_mixed,y_mixed)

y_pred_probs=clf.predict(X_test)
y_pred = np.zeros_like(y_pred_probs)
y_pred[np.arange(len(y_pred_probs)), y_pred_probs.argmax(1)] = 1
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

  

>     Accuracy: 0.9555555555555556

In [ ]:
### Attempt to port the previous example to MNIST
from sklearn.datasets import fetch_openml
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)

In [ ]:
enc = LabelBinarizer()
enc.fit(y)
y = enc.transform(y)

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70
X_train_base = X_train.copy()
y_train_base = y_train.copy()

###
### Create the MixUp data set
# number of MixUp extensions
number_of_mixups = 4
n = np.shape(X_train_base)[0]
for i in range(number_of_mixups):
  shuffled_indices = np.arange(n).tolist()
  np.random.shuffle(shuffled_indices)
  X_s = X_train_base[shuffled_indices]
  y_s = y_train_base[shuffled_indices]
  mixup_l = np.random.beta(0.2,0.2)
  X_mixed = X_train_base*(1-mixup_l) + mixup_l*X_s
  y_mixed = y_train_base*(1-mixup_l) + (mixup_l)*(y_s)
  X_train = np.concatenate([X_train, X_mixed])
  y_train = np.concatenate([y_train, y_mixed])



#Import Random Forest Model
from sklearn.ensemble import RandomForestRegressor
clf=RandomForestRegressor(n_estimators=10)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_mixed,y_mixed)

y_pred_probs=clf.predict(X_test)
y_pred = np.zeros_like(y_pred_probs)
y_pred[np.arange(len(y_pred_probs)), y_pred_probs.argmax(1)] = 1
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

  

>     Accuracy: 0.9771428571428571

In [ ]:
### Attempt to port the previous example to MNIST
#from sklearn.datasets import fetch_openml
#X, y = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False)
import tensorflow as tf
(X, y),(testX,testY) = tf.keras.datasets.fashion_mnist.load_data()
X = X.reshape(60000, 28*28)

enc = LabelBinarizer()
enc.fit(y)
y = enc.transform(y)

  

  

>     Out[285]: (59940,)

In [ ]:
# Fixes the issue "AttributeError: 'ConsoleBuffer has no attribute 'fileno'"
import sys
sys.stdout.fileno = lambda: False

from tune_sklearn import TuneGridSearchCV
import numpy as np

# Split dataset into training set and test set
num_classes = 10
np.random.seed(1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1, test_size=0.99) # 70
X_train_base = X_train.copy()
y_train_base = y_train.copy()

# Create mixup data points
number_of_mixups = 10
n = np.shape(X_train_base)[0]
for i in range(number_of_mixups):
  shuffled_indices = np.arange(n).tolist()
  np.random.shuffle(shuffled_indices)
  X_s = X_train_base[shuffled_indices]
  y_s = y_train_base[shuffled_indices]
  mixup_l = np.random.beta(0.2,0.2)
  X_mixed = X_train_base*(1-mixup_l) + mixup_l*X_s
  y_mixed = y_train_base*(1-mixup_l) + (mixup_l)*(y_s)
  X_train = np.concatenate([X_train, X_mixed])
  y_train = np.concatenate([y_train, y_mixed])

parameters = {
    'n_estimators': range(10,11),
    'max_depth': range(4,5)
}

# n_jobs=-1 enables use of all cores like Tune does
tune_search = TuneGridSearchCV(
    RandomForestRegressor(),
    parameters#, early_stopping="MedianStoppingRule", max_iters=10
)

import time
start = time.time()
hist = tune_search.fit(X_train, y_train)
end = time.time()
print("Sklearn Fit Time:", end - start)
y_pred_probs = tune_search.predict(X_test)
y_pred = np.zeros_like(y_pred_probs)
y_pred[np.arange(len(y_pred_probs)), y_pred_probs.argmax(1)] = 1
#accuracy = np.count_nonzero(np.array(y_pred) == np.array(y_test)) / (num_classes*len(y_pred))
#print("Sklearn Accuracy:", accuracy)
#acc2 = np.mean(np.argmax(y_pred,1) == np.argmax(y_test,1))
#print(acc2)
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy3:",metrics.accuracy_score(y_test, y_pred))

  

>     (pid=12141) /databricks/python/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
>     (pid=12141)   "multioutput='uniform_average').", FutureWarning)
>     (pid=12141) /databricks/python/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
>     (pid=12141)   "multioutput='uniform_average').", FutureWarning)
>     (pid=12141) /databricks/python/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
>     (pid=12141)   "multioutput='uniform_average').", FutureWarning)
>     (pid=12141) /databricks/python/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
>     (pid=12141)   "multioutput='uniform_average').", FutureWarning)
>     (pid=12141) /databricks/python/lib/python3.7/site-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
>     (pid=12141)   "multioutput='uniform_average').", FutureWarning)
>     Sklearn Fit Time: 59.81026768684387
>     Accuracy3: 0.6262794612794613

In [ ]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy3:",metrics.accuracy_score(y_test, y_pred))

  

>     Accuracy3: 0.6641666666666667

In [ ]:
###
### Attempt at a nicer MixUp implementation. Unfortunately I cannot change the labels through this pipeline... Maybe it can be solved

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

# Note: the x-mixup works, but I have not managed to make it work for y

class MixUpTransformer(BaseEstimator, TransformerMixin):
  def __init__(self):
    print('\nhehe1\n')
  def fit(self, x, y):
    print('\nhehe2\n')
    return self
  def transform(self, x, y = None):
    print('\nhehe3\n')
    #y_s = y.copy()
    n = np.shape(x)[0]
    shuffled_indices = np.arange(n).tolist()
    np.random.shuffle(shuffled_indices)
    x_s = x.iloc[shuffled_indices]
    x_mixed = x.mul(0.3).add(x_s).mul(0.7)
    return x_mixed
  
from sklearn.pipeline import make_pipeline
pipe1 = make_pipeline(MixUpTransformer(), RandomForestClassifier())

  

>     hehe1

In [ ]:
pipe1.fit(X_train, y_train)
y_pred=pipe1.predict(X_test)
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

  

>     hehe2

In [ ]:
# Fixes the issue "AttributeError: 'ConsoleBuffer has no attribute 'fileno'"
import sys
sys.stdout.fileno = lambda: False

from tune_sklearn import TuneGridSearchCV
import numpy as np

parameters = {
    'max_depth': range(100)
}

# n_jobs=-1 enables use of all cores like Tune does
tune_search = TuneGridSearchCV(
    RandomForestClassifier(),
    parameters,
    early_stopping="MedianStoppingRule",
    max_iters=10
)

import time
start = time.time()
sklearn_search.fit(X_train, y_train)
end = time.time()
print("Sklearn Fit Time:", end - start)
pred = sklearn_search.predict(X_test)
accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
print("Sklearn Accuracy:", accuracy)

  

>     Sklearn Fit Time: 0.9184603691101074
>     Sklearn Accuracy: 0.9111111111111111

In [ ]:
#from sklearn.model_selection import GridSearchCV
from tune_sklearn import TuneGridSearchCV

# Other imports
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier

# Set training and validation sets
X, y = make_classification(n_samples=11000, n_features=1000, n_informative=50, n_redundant=0, n_classes=10, class_sep=2.5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1000)

# Example parameters to tune from SGDClassifier
parameters = {
    'alpha': [1e-4, 1e-1, 1],
    'epsilon':[0.01, 0.1]
}

tune_search = TuneGridSearchCV(
    SGDClassifier(),
    parameters,
    early_stopping="MedianStoppingRule",
    max_iters=10
)

import time # Just to compare fit times
start = time.time()
tune_search.fit(X_train, y_train)
end = time.time()
print("Tune Fit Time:", end - start)
pred = tune_search.predict(X_test)
accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
print("Tune Accuracy:", accuracy)

  

>     Tune Fit Time: 41.66382598876953
>     Tune Accuracy: 0.88

In [ ]:
from sklearn.model_selection import GridSearchCV
# n_jobs=-1 enables use of all cores like Tune does
sklearn_search = GridSearchCV(
    SGDClassifier(),
    parameters,
    n_jobs=-1
)

import time
start = time.time()
sklearn_search.fit(X_train, y_train)
end = time.time()
print("Sklearn Fit Time:", end - start)
pred = sklearn_search.predict(X_test)
accuracy = np.count_nonzero(np.array(pred) == np.array(y_test)) / len(pred)
print("Sklearn Accuracy:", accuracy)

  

>     Sklearn Fit Time: 82.67133474349976
>     Sklearn Accuracy: 0.876